## 🪄 Install `wandb` library and login


Start by installing the library and logging in to your free account.



In [ ]:
!pip install wandb 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 KB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 5.1 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=f4c9aa992b20bcfc213b0938beeb381518f4b142c573db505ecedff6344fc71a
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [ ]:
# Log in to your W&B account
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
#@title Imports
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pdb
import time


In [ ]:
#@title Load Data
epochs = 30
batch_size = 32
filter_size = 20 #this is the parameter i changed for part 1 - Architecture, number of filters
lr = 0.001

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
#@title Define CNN
import torch.nn as nn
import torch.nn.functional as F


# TO use GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# part 1 - regular network
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, filter_size, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(filter_size, filter_size * 3, 5)
        self.fc1 = nn.Linear(filter_size *3 * 5 * 5, 10)


    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc1(x)
        return x

# part 2 - linear (without non linear layers)   
# class Net(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv2d(3, filter_size, 5)
#         self.conv2 = nn.Conv2d(filter_size, filter_size * 3, 5)
#         self.fc1 = nn.Linear(filter_size *3 * 24 * 24, 10)


#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.conv2(x)
#         x = torch.flatten(x, 1)
#         x = self.fc1(x)
#         return x

# part 3 - Receptive Field
# option 1
# class Net(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv2d(3, filter_size, 5)
#         self.fc1 =nn.Linear(filter_size  * 28 * 28, 10)


#     def forward(self, x):
#         x = F.relu(self.conv1(x))
#         x = torch.flatten(x, 1) # flatten all dimensions except batch
#         x =(self.fc1(x))
#         return x


# option 2

# class Net(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv2d(3, filter_size, 5)
#         self.conv2 = nn.Conv2d(filter_size, filter_size * 3, 5)
#         self.avgpool = nn.AvgPool2d(24,24)  # Global average pooling
#         self.fc1 = nn.Linear(filter_size * 3, 10)

#     def forward(self, x):
#         x = F.relu(self.conv1(x))
#         x = F.relu(self.conv2(x))
#         x = self.avgpool(x)
#         x = torch.flatten(x, 1)  # flatten all dimensions except batch
#         x = self.fc1(x)
#         return x

net = Net().to(device)
total_params = sum(p.numel() for p in net.parameters())
print(f"Total number of parameters: {total_params}")


Total number of parameters: 32190


In [ ]:
#@title Test CNN loss
# prepare to count predictions for each class
def evaluate_loss():
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}
    running_loss = 0
    # again no gradients needed
    with torch.no_grad():
        for data in testloader:
            images, labels =  data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predictions = torch.max(outputs, 1)
            # collect the correct predictions for each class
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1

            loss = criterion(outputs, labels)
            running_loss += loss
        # wandb.log({"Test Loss": running_loss})



    # print accuracy for each class
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')
    test_loss = running_loss / len(testloader)
    return test_loss 


In [ ]:
#@title Test CNN Accuracy
def evaluate_accuracy():

    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            images, labels =data[0].to(device), data[1].to(device)
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        accuracy = ( correct / total) * 100
        # wandb.log({"Test accuracy": accuracy})
        return accuracy

In [ ]:
#@title Train the network & Loss function
# Define loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)


start_time = time.time()

# 🐝 1️⃣ Start a new run to track this script
wandb.init(
    # Set the project where this run will be logged
    project="ex1-nn", 
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    name=f"Model using: epochs: {epochs}, batch size: {batch_size}, lr: {lr}, filter size: {filter_size}", 
    # Track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": epochs,
    })
  
# Train
for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    correct_pred = 0
    total_pred = 0

    # Train 
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        outputs = net(inputs)

        # calculate statistics
        loss = criterion(outputs, labels)
        _, predicted = torch.max(outputs.data, 1)

        # for calculating accuracy and running_loss
        total_pred += labels.size(0)
        correct_pred += (predicted == labels).sum().item()
        running_loss += loss.item()

        
        # forward + backward + optimize
        loss.backward()
        optimizer.step()



    # save train accuracy & loss               
    accuracy = (correct_pred/total_pred) * 100
    train_loss = running_loss / len(trainloader)
    # save test accuracy & loss
    print("##### Epoch " + str(epoch)+ " #####")               
    test_loss = evaluate_loss()
    test_accuracy = evaluate_accuracy()
    wandb.log({"Train Accuracy": accuracy, "Train Loss": train_loss,"Test Accuracy": test_accuracy, "Test Loss": test_loss})




end_time = time.time()
total_time = str(end_time - start_time)

print('Finished Training, it took:' + total_time + ' seconds')

Test Accuracy,▁█
Test Loss,█▁
Train Accuracy,▁█
Train Loss,█▁
Test Accuracy,27.78
Test Loss,1.939
Train Accuracy,25.064
Train Loss,2.00879


torch.Size([40, 3, 32, 32])
torch.Size([40, 20, 28, 28])
torch.Size([40, 60, 24, 24])
torch.Size([40, 60, 1, 1])
torch.Size([40, 60])
torch.Size([40, 10])
torch.Size([40, 3, 32, 32])
torch.Size([40, 20, 28, 28])
torch.Size([40, 60, 24, 24])
torch.Size([40, 60, 1, 1])
torch.Size([40, 60])
torch.Size([40, 10])
torch.Size([40, 3, 32, 32])
torch.Size([40, 20, 28, 28])
torch.Size([40, 60, 24, 24])
torch.Size([40, 60, 1, 1])
torch.Size([40, 60])
torch.Size([40, 10])
torch.Size([40, 3, 32, 32])
torch.Size([40, 20, 28, 28])
torch.Size([40, 60, 24, 24])
torch.Size([40, 60, 1, 1])
torch.Size([40, 60])
torch.Size([40, 10])
torch.Size([40, 3, 32, 32])
torch.Size([40, 20, 28, 28])
torch.Size([40, 60, 24, 24])
torch.Size([40, 60, 1, 1])
torch.Size([40, 60])
torch.Size([40, 10])
torch.Size([40, 3, 32, 32])
torch.Size([40, 20, 28, 28])
torch.Size([40, 60, 24, 24])
torch.Size([40, 60, 1, 1])
torch.Size([40, 60])
torch.Size([40, 10])
torch.Size([40, 3, 32, 32])
torch.Size([40, 20, 28, 28])
torch.Size([4

KeyboardInterrupt: ignored

In [ ]:
wandb.finish()